In [1]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
### Célula para conectar com Google Drive
from google.colab import drive
drive.mount('/content/drive')

if not os.getcwd().endswith('Censo IBGE 2022/Compatibilização'):
    os.chdir('/content/drive/Shareddrives/SIG LabCidade/projetos/Censo IBGE 2022/Compatibilização')

Mounted at /content/drive


In [3]:
nome_compat = '2010-2022-RMSP-RMBS-AUJ'
nome_A = '2010'
nome_B = '2022'

## Importar geometria de distritos

In [4]:
# Importar perímetros compatíveis
gdf_dist = gpd.read_file('../../../dados/IBGE 2022 Agregados preliminares por setores censitários/georreferenciados/SP_Malha_Preliminar_2022_c_tributos.gpkg')
gdf_dist = gdf_dist[['CD_DIST', 'NM_DIST', 'CD_MESO', 'NM_MESO', 'geometry']]
gdf_dist = gdf_dist.dissolve(by=['CD_DIST', 'NM_DIST', 'CD_MESO', 'NM_MESO'])
gdf_dist = gdf_dist.to_crs("EPSG:31983")

## Importar dados IBGE

Dados comuns entre censos:

| Código | Variável | Localização em A | Localização em B |
|--------|----------|------------------|------------------|
| pop_tot | População total | Domicilio02_SP_V001 | v0001 |
| pop_dom_po | População em domicílios particulares | Domicilio02_SP_V002 (apenas permanentes) | v0005b = v0005 * v0007 |
| dom_tot | Domicílios totais | Domicilio01_SP_V001 | v0002 |
| dom_po | Domicílios particulares ocupados | Domicilio01_SP_V002 + DomicilioRenda_SP_V001 | v0007 |

In [5]:
# Dados malha A
loc_variaveis = [
    {
    'Basico_SP1': ['Cod_distrito'],
    'Basico_SP2': ['Cod_distrito'],},
    {
    'Domicilio01_SP1': ['V001', 'V002'],
    'Domicilio01_SP2': ['V001', 'V002'],},
     {
    'Domicilio02_SP1': ['V001', 'V002'],
    'Domicilio02_SP2': ['V001', 'V002'],},
     {
    'DomicilioRenda_SP1': ['V001'],
    'DomicilioRenda_SP2': ['V001'],}
]

data_A = pd.read_csv('2010-2022-RMSP-RMBS-AUJ/matriz_compat_2010.csv', sep='\t', dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})
data_A = data_A.dropna(subset='CD_GEOCODI')
data_A = data_A[['CD_GEOCODI']]

for group in loc_variaveis:
    df_group = pd.DataFrame()
    for file, variaveis in group.items():
        df_temp = pd.read_csv(f'../../../dados/IBGE 2010 Agregados por setores censitários/{file}.csv',
                            sep=';',
                            usecols=['Cod_setor']+variaveis,
                            dtype={'Cod_setor':str,
                                   'Cod_distrito':str},
                            encoding='latin-1')
        for v in variaveis:
            if v.startswith('V'):
                df_temp[v] = df_temp[v].replace('X',0).astype(float)
            else:
                df_temp[v] = df_temp[v].replace('X',0).astype(str)
        df_temp = df_temp.rename(columns={k:f'{file[:-1]}_{k}' for k in variaveis})
        df_temp = df_temp.rename(columns={'Cod_setor':'CD_GEOCODI'})
        df_group = pd.concat([df_group, df_temp])
    data_A = data_A.merge(df_group, on='CD_GEOCODI', how='left')

data_A = data_A.rename(columns={'Basico_SP_Cod_distrito':'CD_DIST'})

In [6]:
# Dados malha B
data_B = pd.read_csv('../../../dados/IBGE 2022 Agregados preliminares por setores censitários/Agregados_preliminares_por_setores_censitarios_SP.csv',
            sep=';',
            dtype={'CD_SETOR':str,
                   'CD_DIST':str,
                   'CD_MESO':str})
data_B = data_B[['CD_DIST', 'NM_DIST', 'CD_MESO', 'NM_MESO', 'v0001', 'v0002', 'v0003', 'v0004', 'v0005', 'v0006', 'v0007']]
data_B = data_B.rename(columns={'CD_SETOR':'CD_GEOCODI'})

## Reagregação

In [7]:
# Malha A
### Agregação
aggfuncs = {k:'sum' for k in data_A.columns if k not in ['CD_DIST', 'CD_GEOCODI']}
agg_A = data_A.pivot_table(index='CD_DIST', values=aggfuncs.keys(), aggfunc=aggfuncs).reset_index()

### Calcular dom_po
agg_A['dom_po'] = agg_A['Domicilio01_SP_V002'] + agg_A['DomicilioRenda_SP_V001']

### Renomear variáveis
agg_A = agg_A.rename(columns={k:f'{nome_A}_{k}' for k in list(agg_A.columns) if k != 'CD_DIST'})

In [8]:
# Malha B
### Moradores em Domicílios particulares ocupados
data_B['v0005b'] = data_B.apply(lambda x: int(round(x['v0005']*x['v0007'])), axis=1)

### Imputados de real para inteiro
data_B['v0006b'] = data_B.apply(lambda x: int(round(x['v0007']*(x['v0006']/100),0)), axis=1)

### Agregação
aggfuncs = {
    'v0001': 'sum',
    'v0002': 'sum',
    'v0003': 'sum',
    'v0004': 'sum',
    'v0005b': 'sum',
    'v0006b': 'sum',
    'v0007': 'sum'
}
agg_B = data_B.pivot_table(index=['CD_DIST', 'NM_DIST', 'CD_MESO', 'NM_MESO'], values=aggfuncs.keys(), aggfunc=aggfuncs).reset_index()

### Recalcular variáveis compostas
agg_B['v0005'] = agg_B['v0005b']/agg_B['v0007']
agg_B['v0006'] = agg_B['v0006b']*100/agg_B['v0007']

### Renomear variáveis
agg_B = agg_B.rename(columns={k:f'{nome_B}_{k}' for k in list(agg_B.columns) if k != 'CD_DIST'})

In [9]:
agg = agg_B.merge(agg_A, on='CD_DIST', how='inner')

## Renomeações

In [10]:
new_names = {'2022_v0001': 'pop_tot_2022',
             '2022_v0002': 'dom_tot_2022',
             '2022_v0003': 'dom_par_2022',
             '2022_v0004': 'dom_col_2022',
             '2022_v0005b': 'pop_dom_po_2022',
             '2022_v0006b': 'dom_po_imput_2022',
             '2022_v0007': 'dom_po_2022',
             '2022_v0005': 'med_pop_dom_po_2022',
             '2022_v0006': 'pct_dom_po_imput_2022',
             '2010_Domicilio01_SP_V001': 'dom_tot_2010',
             '2010_Domicilio01_SP_V002': 'dom_pp_2010',
             '2010_Domicilio02_SP_V001': 'pop_tot_2010',
             '2010_Domicilio02_SP_V002': 'pop_dom_pp_2010',
             '2010_DomicilioRenda_SP_V001': 'dom_pi_2010',
             '2010_dom_po': 'dom_po_2010'}

agg = agg.rename(columns=new_names)
agg = agg[sorted(agg.columns)]

## Cálculos de variação

In [11]:
gdf_agg = gdf_dist.merge(agg, on='CD_DIST', how='left')[[i for i in agg.columns]+['geometry']]

# Variação de população
gdf_agg['var_pop_tot'] = (gdf_agg['pop_tot_2022']-gdf_agg['pop_tot_2010'])

# Variação de domicílios
gdf_agg['var_dom_tot'] = (gdf_agg['dom_tot_2022']-gdf_agg['dom_tot_2010'])

# Variação de domicílios
gdf_agg['var_dom_po'] = (gdf_agg['dom_po_2022']-gdf_agg['dom_po_2010'])

# Variação de densidade populacional
gdf_agg['var_dens_pop_tot_ha'] = gdf_agg['var_pop_tot']/(gdf_agg['geometry'].area/10000)

# Variação de densidade domiciliar (total)
gdf_agg['var_dens_dom_tot_ha'] = gdf_agg['var_dom_tot']/(gdf_agg['geometry'].area/10000)

# Variação de densidade domiciliar (ocupados)
gdf_agg['var_dens_dom_po_ha'] = gdf_agg['var_dom_po']/(gdf_agg['geometry'].area/10000)

# Variação da média de moradores por domicílio
gdf_agg['var_med_pop_dom'] = gdf_agg['med_pop_dom_po_2022']-(gdf_agg['pop_dom_pp_2010']/gdf_agg['dom_pp_2010'])

## Exportação

In [12]:
agg.to_csv(f'{nome_compat}/agg_distritos_{nome_A}-{nome_B}.csv', sep='\t', index=False, decimal=',')

In [13]:
gdf_agg.to_file(f'{nome_compat}/distritos_agg.gpkg',
                      layer=f'{nome_A}-{nome_B}',
                      driver='GPKG')